In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso,Ridge, ElasticNet, BayesianRidge, LinearRegression
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
data = pd.read_csv('person_csvs/all_people.csv',index_col=0)
resp = [i for i in data.columns if 'RESP' in i and i!=' RESP']
data.drop(resp+['Time [s]','sec'],axis=1,inplace=True)

In [3]:
SS = StandardScaler()
X = data.drop(' RESP',axis=1)
SS.fit(X,y=None)
y = data[' RESP'].values
X = SS.transform(X)

In [4]:
models = {
#           'OLS':LinearRegression(),'ElasticNet':ElasticNet(),
#           'BayesianRidge':BayesianRidge(),'Lasso':Lasso(),
#           'Ridge':Ridge(),'KNN':neighbors.KNeighborsRegressor(),
          'rff':RandomForestRegressor(),
#           'ada': AdaBoostRegressor(),
#           'xgb':XGBRegressor()
         }

def model_performance(X,y):
    times =[]
    keys = []
    mean_squared_errors = []
    mean_abs_error = []
    R2_scores = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

    for k,v in tqdm(models.items()):
        model = v
        t0=time.time()
        model.fit(X_train, y_train)
        train_time = time.time()-t0
        t1 = time.time()
        pred = model.predict(X_test)
        predict_time = time.time()-t1
        pred = pd.Series(pred)
        Time_total = train_time+predict_time
        times.append(Time_total)
        R2_scores.append(r2_score(y_test,pred))
        mean_squared_errors.append(mean_squared_error(y_test,pred))
        mean_abs_error.append(mean_absolute_error(y_test,pred))
        keys.append(k)
    table = pd.DataFrame({'model':keys, 'RMSE':mean_squared_errors,'MAE':mean_abs_error,'R2 score':R2_scores,'time':times})
    table['RMSE'] = table['RMSE'].apply(lambda x: np.sqrt(x))
    return table

In [5]:
model_performance(X,y)

100%|██████████| 1/1 [06:14<00:00, 374.63s/it]


,MAE,R2 score,RMSE,model,time
0,0.061049,0.897488,0.122527,rff,374.558783
